In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install holidays
import holidays
from tqdm import tqdm

import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestRegressor

In [ ]:
path='/content/drive/MyDrive/my_data/Gas_Demand' # 파일이 존재하는 디렉토리
train = pd.read_csv(path+"/한국가스공사_시간별 공급량_20181231.csv", encoding = 'CP949')
test = pd.read_csv(path+"/test.csv")
submission = pd.read_csv(path+"/sample_submission.csv")

In [ ]:
def min_max(data):
  data=data.reset_index(drop=True)
  new_data=[]
  for i in range(data.shape[0]):
    new_data.append((data[i]-data.min())/(data.max()-data.min()))
  return new_data

In [ ]:
def preprocess(train):
  group_day=pd.DataFrame()
  data=train.groupby(['구분','연월일']).공급량.sum().values

  day=pd.DataFrame(train.groupby(['Year','Month']).count()['연월일'].values)
  new_day=pd.DataFrame()
  for i in range(7):
    new_day=new_day.append(day)
  new_day.reset_index(drop=True,inplace=True)

  for i in tqdm(range(data.shape[0])):
    group_day=group_day.append(pd.DataFrame(np.repeat(data[i],24)))
    group_day.reset_index(drop=True,inplace=True)
  train['group_day']=group_day
  return train

In [ ]:
test[['연월일','시간','구분']] = test['일자|시간|구분'].str.split(' ',expand=True)
test.drop(['일자|시간|구분'], axis = 1, inplace= True)

In [ ]:
train['연월일'] = pd.to_datetime(train['연월일'])
test['연월일'] = pd.to_datetime(test['연월일'])
train['Month']=pd.DatetimeIndex(train['연월일']).month
test['Month']=pd.DatetimeIndex(test['연월일']).month
train['Year']=pd.DatetimeIndex(train['연월일']).year
test['Year']=pd.DatetimeIndex(test['연월일']).year
train['Weekday']=pd.DatetimeIndex(train['연월일']).weekday
test['Weekday']=pd.DatetimeIndex(test['연월일']).weekday

In [ ]:
train = train[(train['구분'] != 'H') | (train['공급량'] < 1500)]
train = train[(train['구분'] != 'C') | (train['공급량'] > 3)]

In [ ]:
train['하루 가스공급량 시간당 비율'] = 0
for id in ['A', 'B', 'C', 'D', 'E', 'G', 'H']:
    data = train.loc[train.구분==id]
    data['하루 가스공급량 시간당 비율'] = 0
    data.reset_index(drop=True,inplace=True)

    for i in tqdm(range(data.shape[0])):
        data.loc[i,'하루 가스공급량 시간당 비율'] = data.loc[i,'공급량']/sum(data.loc[data.연월일==data.loc[i,'연월일'],'공급량'])
    data.index = train.loc[train.구분==id].index
    train.loc[train.구분==id] = data

100%|██████████| 52579/52579 [01:01<00:00, 855.54it/s]


In [ ]:
data_A=train.loc[train.구분=='A'].reset_index(drop=True)
data_B=train.loc[train.구분=='B'].reset_index(drop=True)
data_C=train.loc[train.구분=='C'].reset_index(drop=True)
data_D=train.loc[train.구분=='D'].reset_index(drop=True)
data_E=train.loc[train.구분=='E'].reset_index(drop=True)
data_G=train.loc[train.구분=='G'].reset_index(drop=True)
data_H=train.loc[train.구분=='H'].reset_index(drop=True)

In [ ]:
new_train=pd.DataFrame()
data=preprocess(data_A)
new_train=new_train.append(data)
data=preprocess(data_B)
new_train=new_train.append(data)
data=preprocess(data_C)
new_train=new_train.append(data)
data=preprocess(data_D)
new_train=new_train.append(data)
data=preprocess(data_E)
new_train=new_train.append(data)
data=preprocess(data_G)
new_train=new_train.append(data)
data=preprocess(data_H)
new_train=new_train.append(data)
new_train.reset_index(drop=True,inplace=True)

100%|██████████| 2191/2191 [00:01<00:00, 1720.01it/s]


In [ ]:
new_train.to_csv(path+"/final_data_129.csv",index=False)

In [ ]:
new_train=pd.read_csv(path+"/final_data_129.csv")
new_train

,연월일,시간,구분,공급량,Month,Year,Weekday,하루 가스공급량 시간당 비율,group_day
0,2013-01-01,1,A,2497.129,1,2013,1,0.041532,60125.336
1,2013-01-01,2,A,2363.265,1,2013,1,0.039306,60125.336
2,2013-01-01,3,A,2258.505,1,2013,1,0.037563,60125.336
3,2013-01-01,4,A,2243.969,1,2013,1,0.037322,60125.336
4,2013-01-01,5,A,2344.105,1,2013,1,0.038987,60125.336
...,...,...,...,...,...,...,...,...,...
365323,2018-12-31,20,H,681.033,12,2018,0,0.047997,14189.118
365324,2018-12-31,21,H,669.961,12,2018,0,0.047217,14189.118
365325,2018-12-31,22,H,657.941,12,2018,0,0.046369,14189.118
365326,2018-12-31,23,H,610.953,12,2018,0,0.043058,14189.118


In [ ]:
kr_holidays = holidays.KR()
new_train['holiday'] = new_train['연월일'].apply(lambda x: True if x in kr_holidays else False)
test['holiday'] = test['연월일'].apply(lambda x: True if x in kr_holidays else False)

#일별 공급량 예측

In [ ]:
number=[]
for n in range(int(new_train.shape[0]/24)):
  for num in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]:
    number.append(num)
new_train['num']=number

number=[]
for n in range(int(test.shape[0]/24)):
  for num in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]:
    number.append(num)
test['num']=number

In [ ]:
day_train=new_train.loc[new_train.num==24]
day_test=test.loc[test.num==24]
day_train.reset_index(drop=True,inplace=True)
day_test.reset_index(drop=True,inplace=True)

In [ ]:
day_train = pd.get_dummies(day_train, columns = ['구분'])
day_test = pd.get_dummies(day_test, columns = ['구분'])

In [ ]:
_date=day_test.연월일

In [ ]:
day_train_label = np.log1p(day_train['group_day'])
day_train.drop(['연월일','시간','공급량','Year','group_day','하루 가스공급량 시간당 비율','num'], axis = 1, inplace= True)
day_test.drop(['연월일','시간','Year','num'], axis = 1, inplace= True)

In [ ]:
rf = RandomForestRegressor(
                n_estimators=100,
                criterion='squared_error',
                max_features='auto',
                bootstrap=True,
                random_state=0,
                n_jobs=-1,
)

In [ ]:
rf.fit(day_train,day_train_label)
day_pred = np.expm1(rf.predict(day_test))
day_pred_train = np.expm1(rf.predict(day_train))

#시간별 비율 예측

In [ ]:
ratio_train = pd.get_dummies(new_train, columns = ['구분'])
ratio_test = pd.get_dummies(test, columns = ['구분'])

In [ ]:
ratio_train_label=ratio_train['하루 가스공급량 시간당 비율']
ratio_train.drop(['연월일','공급량','Year','group_day','하루 가스공급량 시간당 비율','num'], axis = 1, inplace= True)
ratio_test.drop(['연월일','Year','num'], axis = 1, inplace= True)

In [ ]:
rf = RandomForestRegressor(
                n_estimators=100,
                criterion='squared_error',
                max_features='auto',
                bootstrap=True,
                random_state=0,
                n_jobs=-1,
)
rf.fit(ratio_train,ratio_train_label)
ratio_pred = rf.predict(ratio_test)

In [ ]:
ratio_pred=pd.DataFrame({'연월일':test.연월일,'비율':ratio_pred})

In [ ]:
ratio_pred

,연월일,비율
0,2019-01-01,0.041079
1,2019-01-01,0.039022
2,2019-01-01,0.037177
3,2019-01-01,0.037018
4,2019-01-01,0.038212
...,...,...
15115,2019-03-31,0.055343
15116,2019-03-31,0.056485
15117,2019-03-31,0.051882
15118,2019-03-31,0.046230


In [ ]:
day_pred=pd.DataFrame({'연월일':_date,'하루공급량':day_pred})

In [ ]:
count=0
cou=0
for i in range(ratio_pred.shape[0]):
  ratio_pred.iloc[i,1]=ratio_pred.iloc[i,1]*day_pred.iloc[count,1]
  cou=cou+1
  if(cou==24):
    count=count+1
    cou=0

In [ ]:
submission['공급량'] = ratio_pred.비율
submission.to_csv(path+'/최종제출.csv',index=False)

In [ ]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,2280.658765
1,2019-01-01 02 A,2166.488190
2,2019-01-01 03 A,2064.032317
3,2019-01-01 04 A,2055.193923
4,2019-01-01 05 A,2121.518204
...,...,...
15115,2019-03-31 20 H,413.828151
15116,2019-03-31 21 H,422.368344
15117,2019-03-31 22 H,387.946256
15118,2019-03-31 23 H,345.680738
